# Model combining Surface energy + Heat equation + Boundary layer

In [ ]:
import numpy as np
import math
from scipy.optimize import minimize, minimize_scalar
import matplotlib.pyplot as plt
import plotly.express as px
from time import perf_counter
from tqdm import tqdm
import pickle

In [ ]:
def heat_equation_time(depth, length, Nz, Nx, years):
    """ This is an example of an time-dependent heat equation using a 
    sinus wave temperature signal at the surface. The heat equation is solved for a 
    pre-defined number of years over the domain depth using Nz grid points."""

    # Definitions and assignments
    integration = 365*years    # Integration time in days
    dz  = depth/Nz             # Distance between grid points
    dx  = length/Nx             # Distance between grid points
    dt  = 86400                # Time step in seconds (for each day)
    K   = 1.2e-6               # Conductivity
 
    # Define index arrays 
    idx_z  = np.arange(1,Nz-1)  # all indices at location i
    idx_r_l  = np.arange(0,Nz-2) # all indices at location i-1
    idx_z_r  = np.arange(2,Nz)   # all indices at location i+1
    
    idx_x = np.arange(1,Nx-1)
    idx_x_l = np.arange(0,Nx-2)
    idx_x_r = np.arange(2,Nx)

    # Initial temperature field
    T = np.zeros((Nz, Nx))

    # Create array for new temperature values
    Tnew = T

    # 3D-Array containing the vertical profiles for all time steps (depth, time)
    T_all = np.zeros((Nz, Nx, integration))

    
    # Time loop
    for t in range(integration):
    
        # Set top BC - Dirlichet condition
        T[0] = 10 - 20 * np.sin(2*np.pi*t/365)

        # Set lower BC - Neumann condition
        T[-1] = T[-2]
        
        # Update temperature using indices arrays
        Tnew[k] = T[k] + ((T[kr] - 2*T[k] + T[kl])/dz**2) * dt * K
        
        # Copy the new temperature als old timestep values (used for the 
        # next time loop step)
        T[1:-1] = Tnew[1:-1].copy()

        # Write result into the final array
        T_all[:, t] = T


    # return temperature array, grid spacing, and number of integration steps
    return T_all, dz, integration

In [ ]:

    
# --------------------------
# Auxiliary functions
# --------------------------
def saturation_water_vapor(T):
    """ Calculates the saturation water vapor pressure [Pa]"""
    return ( 6.122*np.exp( (17.67*(T-273.16))/(T-29.66) ) )

def hypsometric_eqn(p0, Tv, z):
    """Hypsometric equation to calculate the pressure at a certain height 
       when the surface pressure is given
       p0 :: surface pressure [hPa]
       Tv :: mean virtual temperature of atmosphere [K]
       z  :: height above ground [m]
    """
    return(p0/(np.exp((9.81*z)/(287.4*Tv) )))

def mixing_ratio(theta, p0, Tv, z):
    """ Calculates the mixing ratio from
        theta :: temperature [K]
        p0    :: surface pressure [hPa]
        Tv    :: mean virtual temperature of atmosphere [K]
        z     :: height [m]
    """
    return(622.97 * (saturation_water_vapor(theta)/(hypsometric_eqn(p0,Tv,z)-saturation_water_vapor(theta))))
           
    
    
    
def boundary_layer_evolution_moisture_gamma(u, K, dx, dz, Nx, Nz, hours, dt):
    """ Simple advection-diffusion equation.
    
    integration :: Integration time in seconds
    Nz          :: Number of grid points
    dt          :: time step in seconds
    K           :: turbulent diffusivity
    u           :: Speed of fluid
    """
       
    # Some definitions
    # Multiply the hours given by the user by 3600 s to 
    # get the integration time in seconds
    integration = hours*3600
    
    # Define index arrays 
    # Since this a 2D problem we need to define two index arrays.
    # The first set of index arrays is used for indexing in x-direction. This
    # is needed to calculate the derivatives in x-direction (advection)
    k   = np.arange(1,Nx-1) # center cell
    kr  = np.arange(2,Nx)   # cells to the right
    kl  = np.arange(0,Nx-2) # cells to the left
    
    # The second set of index arrays is used for indexing in z-direction. This
    # is needed to calculate the derivates in z-direction (turbulent diffusion)
    m   = np.arange(1,Nz-1) # center cell
    mu  = np.arange(2,Nz)   # cells above 
    md  = np.arange(0,Nz-2) # cells below

    # Make height grid
    height = np.array([np.arange(0,Nz*dz,dz),] * Nx).transpose()
    
    # Make lapse rate array
    Gamma = -0.01 * np.ones((Nz, Nx))
    
    # Lake definition (grid points)
    lake_from = 50
    lake_to = 150
    
    # --------------------------
    # Initial temperature field
    # --------------------------
    # Neutral stratification with lapse rate of 0.01 K/m
    # Create a 1D-array with the vertical temperature distribution
    # Surface = 268 K, decreasing according to the dry-adiabative lapse rate 0.01 K/m
    lapse_rate = -0.01
    theta_vec = np.array([268 + lapse_rate * (dz * z) for z in range(Nz)])
    theta = np.array([theta_vec,] * Nx).transpose() 
    
    # The lower temperature boundary needs to be updated where there is the lake
    # Here, were set the temperature at the lower boundary from the grid cell 50
    # to 150 to a temperature of 278 K
    theta[0, lake_from:lake_to] = 278
    
    # --------------------------
    # Initialize moisture fields 
    # --------------------------
    # Init moisture array with a relative humidity of 70 %
    qsat = mixing_ratio(theta, 1013, 270, height)
    
    # Multiply with relative humidity (80 %)
    q = (qsat.T * np.linspace(0.7, 0.2, Nz)).T
    
    
    # The lower moisture boundary needs to be updated where there is the lake
    # Here, were set the moisture at the lower boundary from the grid cell 50
    # to 150 to a mixing ratio of 0.9 times the saturation mixing ratio
    q[0, lake_from:lake_to] = 0.9 * qsat[0, lake_from:lake_to] 

    # --------------------------
    # Init other arrays
    # --------------------------
    cov = np.zeros((Nz, Nx))        # Empty array for the covariances
    adv = np.zeros((Nz, Nx))        # Empty array for the advection term 
    
    # --------------------------
    # Dimensionless parameters
    # --------------------------
    c = (u*dt)/dx
    d = (K*dt)/(dz**2)

    # --------------------------
    # Integrate the model
    # --------------------------
    for idx in range(int(integration/dt)):

        # Set BC top (Neumann condition)
        # The last term accounts for the fixed gradient of 0.01
        theta[Nz-1, :] = theta[Nz-2, :]# - 0.005 * dz
        
        # Set top BC for moisture
        q[Nz-1, :] = q[Nz-2, :] 
        
        # Set BC right (Dirichlet condition)
        theta[:, Nx-1] = theta[:, Nx-2]
        
        # Set right BC for moisture
        q[:, Nx-1] = q[:, Nx-2]
        
        # We need to keep track of the old values for calculating the new derivatives.
        # That means, the temperature value a grid cell is calculated from its values 
        # plus the correction term calculated from the old values. This guarantees that
        # the gradients for the x an z direction are based on the same old values.
        old = theta
        old_q = q
            
        # First update grid cells in z-direction. Here, we loop over all x grid cells and
        # use the index arrays m, mu, md to calculate the gradients for the
        # turbulent diffusion (which only depends on z)
        for x in range(1,Nx-1):
            # Temperature diffusion + lapse rate 
            theta[m,x] = theta[m,x] + ((K*dt)/(dz**2))*(old[mu,x]+old[md,x]-2*old[m,x]) + Gamma[m,x]
            # Turbulent diffusion of moisture
            q[m,x] = q[m,x] + ((K*dt)/(dz**2))*(old_q[mu,x]+old_q[md,x]-2*old_q[m,x])
            # Calculate the warming rate [K/s] by covariance
            cov[m,x] = ((K)/(dz**2))*(old[mu,x]+old[md,x]-2*old[m,x])

        # Then update grid cells in x-direction. Here, we loop over all z grid cells and
        # use the index arrays k, kl, kr to calculate the gradients for the
        # advection (which only depends on x)
        for z in range(1,Nz-1):
            # temperature advection
            theta[z,k] = theta[z,k] - ((u*dt)/(dx))*(old[z,k]-old[z,kl])
            # moisture advection
            q[z,k] = q[z,k] - ((u*dt)/(dx))*(old_q[z,k]-old_q[z,kl])
            # Calculate the warming rate [K/s] by the horizontal advection 
            # Note: Here, we use a so-called upwind-scheme (backward discretization)
            adv[z,k] = - (u/dx)*(old[z,k]-old[z,kl])
            
        # Calculate new saturation mixing ratio
        qsat = mixing_ratio(theta, 1013, 270, height)
        
        # Then the relative humidity using qsat
        rH = np.minimum(q/qsat, 1)
        
        # Correct lapse rates where rH==100% (moist adiabatic lapse rate)
        Gamma[rH==1] = -0.006
        
    # Return results    
    return theta, q, qsat, rH, cov, adv, c, d, np.arange(0, Nx*dx, dx), np.arange(0, Nz*dz, dz)

In [ ]:
class Combined_model:
    
    def __init__(self, Nx, Nz_atmo, Nz_soil, x_size, z_size_atmo, z_size_soil, z, z_0, time, dt):
        
        self.time = time                        # how long to run the model for
        self.dt = dt                            # time step for the model
        self.n_iters = int(time // dt)          # number of iterations

        
        self.Nx = Nx                            # Number of points in x direction
        self.Nz_atmo = Nz_atmo                  # Number of points in z direction in the atmosphere
        self.Nz_soil = Nz_soil                  # Number of points in z direction in the soil
        
        self.dz_atmo  = z_size_atmo/Nz_atmo     # Distance between z grid points in atmo
        self.dz_soil  = z_size_soil/Nz_soil     # Distance between z grid points in soil
        self.dx       = x_size/Nx               # Distance between x grid points
        
        self.time_arr = np.arange(0, time, dt)  # array of times
        
        # 3D Temperature arrays - (z,x,t)
        self.lapse_rate = -0.01
        self.T_atmo_vec = np.array([268 + self.lapse_rate * (self.dz_atmo * z) for z in range(self.Nz_atmo)])
        self.T_atmo     = np.array([[self.T_atmo_vec,]*self.Nx,]*self.n_iters).T
        
        self.T_soil = 240 * np.ones((Nz_soil, Nx, self.n_iters))
        
        # Constants        
        # Atmosphere module
        self.u = 5                 # speed of fluid
        self.atmo_K = .2           # Turbulent diffusivity
        
        # Soil module
        self.soil_K   = 1.2e-6     # Conductivity

        # Surface energy balance module
        self.c_p = 1004.           # specific heat [J kg^-1 K^-1]
        self.kappa = 0.41          # Von Karman constant [-]
        self.sigma = 5.67e-8       # Stefan-Bolzmann constant
        self.L = 2.83e6            # latent heat for sublimation
        
        self.z = z                 # Height for temperature measurement - 2 m
        self.z_0 = z_0             # Surface roughness

    def _init_index_arrs(self):
        self.idx_atmo = {
            'z':   np.arange(1, self.Nz_atmo-1),
            'z_d': np.arange(0, self.Nz_atmo-2),
            'z_u': np.arange(2, self.Nz_atmo),
            }
        self.idx_soil = {
            'z':   np.arange(1, self.Nz_soil-1),
            'z_d': np.arange(0, self.Nz_soil-2),
            'z_u': np.arange(2, self.Nz_soil),
            }
        self.idx_len = {
            'x':   np.arange(1, self.Nx-1),
            'x_l': np.arange(0, self.Nx-2),
            'x_r': np.arange(2, self.Nx),
            }


    def _init_surface(self):
        """Initialises surface parameters"""
        
        self.surf_T_now = np.zeros(self.Nx)      # surface temp at a timestep
        
        # add spatial variability
        self.albedo = 0.3 * np.ones(self.Nx)     # albedo
        self.surf_f = 0.7 * np.ones(self.Nx)     # Relative humidity
        self.surf_rho = 1.1 * np.ones(self.Nx)   # Air density
        self.surf_U = 2.0 * np.ones(self.Nx)     # Wind velocity
        self.surf_z_0 = 1e-3 * np.ones(self.Nx)  # surface Roughness length
        self.surf_p = 1013 * np.ones(self.Nx)    # Pressure
        
        self.Cs_t = self.kappa**2 / (np.log(self.z/self.surf_z_0)**2)
        self.Cs_q = self.Cs_t
        
        # add temporal variability
        #self.surf_G = 700.0 * np.ones((self.Nx, self.n_iters))   # Incoming shortwave radiation
        self.surf_G = 400.0 + 150 * np.sin(2 * np.pi * self.time_arr / (24 * 3600))
    
    def _init_atmosphere(self):
        """Initialises atmospheric parameters"""
        # height arr
        self.height = np.array([np.arange(0,self.dz_atmo*self.Nz_atmo,self.dz_atmo),] * self.Nx).transpose()
        # lapse rate array
        self.lapse_rate_arr = self.lapse_rate * np.ones((self.Nz_atmo, self.Nx))
        # Init moisture array with a relative humidity of 70 %
        self.qsat = self._mixing_ratio(self.T_atmo[:,:,0], 1013, 270, self.height)
        # Multiply with relative humidity (80 %)
        self.q = (self.qsat.T * np.linspace(0.7, 0.2, self.Nz_atmo)).T
        self.cov = np.zeros((self.Nz_atmo, self.Nx))        # Empty array for the covariances
        self.adv = np.zeros((self.Nz_atmo, self.Nx))        # Empty array for the advection term 
        # Dimensionless parameters
        self.c = (self.u*self.dt)/self.dx
        self.d = (self.atmo_K*self.dt)/(self.dz_atmo**2)
        

    def _sat_water_vapor(self, T):
        """ Calculates the saturation water vapor pressure [Pa]"""
        Ew = 6.112 * np.exp((17.67*(T-273.16)) / ((T-29.66)))
        return Ew

    def _hypsometric_eqn(self, p0, Tv, z):
        """Hypsometric equation to calculate the pressure at a certain height 
            when the surface pressure is given
            p0 :: surface pressure [hPa]
            Tv :: mean virtual temperature of atmosphere [K]
            z  :: height above ground [m]
        """
        return(p0/(np.exp((9.81*z)/(287.4*Tv) )))

    def _mixing_ratio(self, theta, p0, Tv, z):
        """ Calculates the mixing ratio from
            theta :: temperature [K]
            p0    :: surface pressure [hPa]
            Tv    :: mean virtual temperature of atmosphere [K]
            z     :: height [m]
        """
        return(622.97 * (self._sat_water_vapor(theta)/(self._hypsometric_eqn(p0,Tv,z)-self._sat_water_vapor(theta))))

    def surface_balance(self, T_a):
        def EB_fluxes(T_0,T_a,f,albedo,G,p,rho,U_L,Cs_t,Cs_q):
            """ This function calculates the energy fluxes"""
            # Correction factor for incoming longwave radiation
            eps_cs = 0.23 + 0.433 * np.power(100*(f*self._sat_water_vapor(T_a))/T_a, 1.0/8.0)

            # Calculate turbulent fluxes
            H_0 = rho * self.c_p * Cs_t * U_L * (T_0 - T_a)
            E_0 = rho * self.L*0.622/p * Cs_q * U_L * self._sat_water_vapor(T_0)*(1 - f)

            # Calculate radiation budget
            L_d = eps_cs * self.sigma * T_a**4
            L_u = 1.0 * self.sigma * T_0**4
            Q_0 = (1-albedo) * G

            return (Q_0, L_d, L_u, H_0, E_0)


        def optim_T0(x,T_a,f,albedo,G,p,rho,U_L,Cs_t,Cs_q):
            """ Optimization function for surface temperature:

            Input: 
            T_0       : Surface temperature, which is optimized [K]
            T_a       : Air tempearature at height self.z
            """
                
            Q_0, L_d, L_u, H_0, E_0 = EB_fluxes(x,T_a,f,albedo,G,p,rho,U_L,Cs_t,Cs_q)
    
            # Get residual for optimization
            res = np.abs(Q_0+L_d-L_u-H_0-E_0)

            # return the residuals
            return res

        # optimise temperature in each point in the x direction

        for x in range(len(self.surf_T_now)):
            T_0 = 283. # temperature which to optimise from
            surf_args = (T_a[x],         self.surf_f[x], self.albedo[x],
                         self.surf_G[self.iter_id], self.surf_p[x], self.surf_rho[x],
                         self.surf_U[x], self.Cs_t[x],   self.Cs_q[x],
                        )
                
            res = minimize(optim_T0,x0=T_0,args=surf_args,bounds=((None,400),), \
                         method='L-BFGS-B',options={'eps':1e-8})
            
            T_0 = res.x[0]

            self.surf_T_now[x] = res.x[0]
            
        return self.surf_T_now

    def boundary_layer(self, T):
        """ Simple advection-diffusion equation.
        Nz          :: Number of grid points
        dt          :: time step in seconds
        K           :: turbulent diffusivity
        u           :: Speed of fluid
        """
        # Set BC top (Neumann condition)
        # The last term accounts for the fixed gradient of 0.01
        T[-1, :] = T[-2, :] - 0.005 * self.dz_atmo

        # Set top BC for moisture
        self.q[-1, :] = self.q[-2, :] 

        # Set BC right (Dirichlet condition)
        T[:, -1] = T[:, -2]

        # Set right BC for moisture
        self.q[:, -1] = self.q[:, -2]
        
        old_t = T
        old_q = self.q

        # First update grid cells in z-direction. Here, we loop over all x grid cells and
        # use the index arrays m, mu, md to calculate the gradients for the
        # turbulent diffusion (which only depends on z)
        for x in range(self.Nx):
            # Temperature diffusion + lapse rate 
            T[self.idx_atmo['z'],x] = T[self.idx_atmo['z'],x] + \
                ((self.atmo_K*self.dt)/(self.dz_atmo**2)) * \
                (old_t[self.idx_atmo['z_u'],x]+old_t[self.idx_atmo['z_d'],x]-2*old_t[self.idx_atmo['z'],x]) + \
                self.lapse_rate_arr[self.idx_atmo['z'],x]
            # Turbulent diffusion of moisture
            self.q[self.idx_atmo['z'],x] = self.q[self.idx_atmo['z'],x] + \
                ((self.atmo_K*self.dt)/(self.dz_atmo**2)) * (old_q[self.idx_atmo['z_u'],x]+old_q[self.idx_atmo['z_d'],x]-2*old_q[self.idx_atmo['z'],x])
            # Calculate the warming rate [K/s] by covariance
            self.cov[self.idx_atmo['z'],x] = ((self.atmo_K)/(self.dz_atmo**2)) * (old_t[self.idx_atmo['z_u'],x] + old_t[self.idx_atmo['z_d'],x] - 2*old_t[self.idx_atmo['z'],x])

        # Then update grid cells in x-direction. Here, we loop over all z grid cells and
        # use the index arrays k, kl, kr to calculate the gradients for the
        # advection (which only depends on x)
        for z in range(1,self.Nz_atmo-1):
            # temperature advection
            T[z,self.idx_len['x']] = T[z,self.idx_len['x']] - \
                ((self.u*self.dt)/(self.dx)) * \
                (old_t[z,self.idx_len['x']]-old_t[z,self.idx_len['x_l']])
            # moisture advection
            self.q[z,self.idx_len['x']] = self.q[z,self.idx_len['x']] - \
                ((self.u*self.dt)/(self.dx)) * (old_q[z,self.idx_len['x']]-old_q[z,self.idx_len['x_l']])
            
            # Calculate the warming rate [K/s] by the horizontal advection 
            # Note: Here, we use a so-called upwind-scheme (backward discretization)
            #adv[z,k] = - (u/dx)*(old[z,k]-old[z,kl])

        # Calculate new saturation mixing ratio
        self.qsat = self._mixing_ratio(T, 1013, 270, self.height)

        # Then the relative humidity using qsat
        rH = np.minimum(self.q/self.qsat, 1)

        # Correct lapse rates where rH==100% (moist adiabatic lapse rate)
        self.lapse_rate_arr[rH==1] = -0.006

        return T
    
    def heat_equation(self, T):
        # Set lower BC - Neumann condition
        T[0, :] = T[1, :]
            
        for x in range(1,self.Nx):
            # Update temperature using indices arrays
            T[self.idx_soil['z'], x] = T[self.idx_soil['z'], x] + \
                ((T[self.idx_soil['z_u'], x] - 2 * T[self.idx_soil['z'], x] + \
                T[self.idx_soil['z_d'], x])/self.dz_soil**2) * self.dt * self.soil_K

        return T
        
    def step(self):

        self.T_atmo[:,:,self.iter_id+1] = self.boundary_layer(self.T_atmo[:, :, self.iter_id])
        self.T_soil[:,:,self.iter_id+1] = self.heat_equation(self.T_soil[:, :, self.iter_id])
        
        # apply surface balance equation
        self.T_atmo[0, :, self.iter_id+1] = self.surface_balance(
            self.T_atmo[0, :, self.iter_id])
        self.T_soil[-1,:, self.iter_id+1] = self.T_atmo[0, :, self.iter_id+1]

    def run(self, verbose=False):

        a = perf_counter()
        
        self._init_index_arrs()
        self._init_surface()
        self._init_atmosphere()
        
        for idx in tqdm(range(self.n_iters-1)):
            if verbose:
                print(f'Epoch # {idx+1}/{self.n_iters}')
            self.iter_id = idx
            self.step()

        print(f'Model run finished in {perf_counter() - a} seconds.')
    
    def show_plot(self):
        concat_arr = np.concatenate((self.T_soil, 272.15*np.ones((1, self.Nx, self.n_iters)),
                                     self.T_atmo), axis=0) - 272.15
        
        fig = px.imshow(np.moveaxis(concat_arr[::-1,:,:], 2, 0), animation_frame=0,
                color_continuous_scale='RdBu_r', aspect='equal', zmin=-30, zmax=30)
        fig.show()


In [ ]:
class Combined_model_numpy:
    
    def __init__(self, Nx, Nz_atmo, Nz_soil, x_size, z_size_atmo, z_size_soil, z, z_0, time, dt):
        
        self.time = np.array([time])                        # how long to run the model for
        self.dt = np.array([dt])                            # time step for the model
        self.n_iters = np.array([int(time // dt)])          # number of iterations

        
        self.Nx = np.array([Nx])                            # Number of points in x direction
        self.Nz_atmo = np.array([Nz_atmo])                  # Number of points in z direction in the atmosphere
        self.Nz_soil = np.array([Nz_soil])                  # Number of points in z direction in the soil
        
        self.dz_atmo  = np.array([z_size_atmo/Nz_atmo])     # Distance between z grid points in atmo
        self.dz_soil  = np.array([z_size_soil/Nz_soil])     # Distance between z grid points in soil
        self.dx       = np.array([x_size/Nx])               # Distance between x grid points
        
        self.time_arr = np.arange(0, time, dt)  # array of times
        
        # 3D Temperature arrays - (z,x,t)
        self.lapse_rate = np.array([-0.01])
        self.T_atmo_vec = np.array([268 + self.lapse_rate * (self.dz_atmo * z) for z in range(self.Nz_atmo)])
        self.T_atmo     = np.array([[self.T_atmo_vec,]*self.Nx,]*self.n_iters).T
        
        self.T_soil = np.ones((Nz_soil, Nx, self.n_iters))
        
        # Constants        
        # Atmosphere module
        self.u = np.array([5])                 # speed of fluid
        self.atmo_K = np.array([.2])           # Turbulent diffusivity
        
        # Soil module
        self.soil_K   = np.array([1.2e-6])     # Conductivity

        # Surface energy balance module
        self.c_p = np.array([1004.])           # specific heat [J kg^-1 K^-1]
        self.kappa = np.array([0.41])          # Von Karman constant [-]
        self.sigma = np.array([5.67e-8])       # Stefan-Bolzmann constant
        self.L = np.array([2.83e6])            # latent heat for sublimation
        
        self.z = np.array([z])                 # Height for temperature measurement - 2 m
        self.z_0 = np.array([z_0])             # Surface roughness

    def _init_index_arrs(self):
        self.idx_atmo = {
            'z':   np.arange(1, self.Nz_atmo-1),
            'z_d': np.arange(0, self.Nz_atmo-2),
            'z_u': np.arange(2, self.Nz_atmo),
            }
        self.idx_soil = {
            'z':   np.arange(1, self.Nz_soil-1),
            'z_d': np.arange(0, self.Nz_soil-2),
            'z_u': np.arange(2, self.Nz_soil),
            }
        self.idx_len = {
            'x':   np.arange(1, self.Nx-1),
            'x_l': np.arange(0, self.Nx-2),
            'x_r': np.arange(2, self.Nx),
            }


    def _init_surface(self):
        """Initialises surface parameters"""
        
        self.surf_T_now = np.zeros(self.Nx)      # surface temp at a timestep
        
        # add spatial variability
        self.albedo = 0.3 * np.ones(self.Nx)     # albedo
        self.surf_f = 0.7 * np.ones(self.Nx)     # Relative humidity
        self.surf_rho = 1.1 * np.ones(self.Nx)   # Air density
        self.surf_U = 2.0 * np.ones(self.Nx)     # Wind velocity
        self.surf_z_0 = 1e-3 * np.ones(self.Nx)  # surface Roughness length
        self.surf_p = 1013 * np.ones(self.Nx)    # Pressure
        
        self.Cs_t = self.kappa**2 / (np.log(self.z/self.surf_z_0)**2)
        self.Cs_q = self.Cs_t
        
        # add temporal variability
        #self.surf_G = 700.0 * np.ones((self.Nx, self.n_iters))   # Incoming shortwave radiation
        self.surf_G = 490.0 + 200 * np.sin(2 * np.pi * self.time_arr / (24 * 3600))
    
    def _init_atmosphere(self):
        """Initialises atmospheric parameters"""
        # height arr
        self.height = np.array([np.arange(0,self.dz_atmo*self.Nz_atmo,self.dz_atmo),] * self.Nx).transpose()
        # lapse rate array
        self.lapse_rate_arr = self.lapse_rate * np.ones((self.Nz_atmo, self.Nx))
        # Init moisture array with a relative humidity of 70 %
        self.qsat = self._mixing_ratio(self.T_atmo[:,:,0], 1013, 270, self.height)
        # Multiply with relative humidity (80 %)
        self.q = (self.qsat.T * np.linspace(0.7, 0.2, self.Nz_atmo)).T
        self.cov = np.zeros((self.Nz_atmo, self.Nx))        # Empty array for the covariances
        self.adv = np.zeros((self.Nz_atmo, self.Nx))        # Empty array for the advection term 
        # Dimensionless parameters
        self.c = (self.u*self.dt)/self.dx
        self.d = (self.atmo_K*self.dt)/(self.dz_atmo**2)
        

    def _sat_water_vapor(self, T):
        """ Calculates the saturation water vapor pressure [Pa]"""
        Ew = 6.112 * np.exp((17.67*(T-273.16)) / ((T-29.66)))
        return Ew

    def _hypsometric_eqn(self, p0, Tv, z):
        """Hypsometric equation to calculate the pressure at a certain height 
            when the surface pressure is given
            p0 :: surface pressure [hPa]
            Tv :: mean virtual temperature of atmosphere [K]
            z  :: height above ground [m]
        """
        return(p0/(np.exp((9.81*z)/(287.4*Tv) )))

    def _mixing_ratio(self, theta, p0, Tv, z):
        """ Calculates the mixing ratio from
            theta :: temperature [K]
            p0    :: surface pressure [hPa]
            Tv    :: mean virtual temperature of atmosphere [K]
            z     :: height [m]
        """
        return(622.97 * (self._sat_water_vapor(theta)/(self._hypsometric_eqn(p0,Tv,z)-self._sat_water_vapor(theta))))

    def surface_balance(self, T_a):
        def EB_fluxes(T_0,T_a,f,albedo,G,p,rho,U_L,Cs_t,Cs_q):
            """ This function calculates the energy fluxes"""
            # Correction factor for incoming longwave radiation
            eps_cs = 0.23 + 0.433 * np.power(100*(f*self._sat_water_vapor(T_a))/T_a, 1.0/8.0)

            # Calculate turbulent fluxes
            H_0 = rho * self.c_p * Cs_t * U_L * (T_0 - T_a)
            E_0 = rho * self.L*0.622/p * Cs_q * U_L * self._sat_water_vapor(T_0)*(1 - f)

            # Calculate radiation budget
            L_d = eps_cs * self.sigma * T_a**4
            L_u = 1.0 * self.sigma * T_0**4
            Q_0 = (1-albedo) * G

            return (Q_0, L_d, L_u, H_0, E_0)


        def optim_T0(x,T_a,f,albedo,G,p,rho,U_L,Cs_t,Cs_q):
            """ Optimization function for surface temperature:

            Input: 
            T_0       : Surface temperature, which is optimized [K]
            T_a       : Air tempearature at height self.z
            """
                
            Q_0, L_d, L_u, H_0, E_0 = EB_fluxes(x,T_a,f,albedo,G,p,rho,U_L,Cs_t,Cs_q)
    
            # Get residual for optimization
            res = np.abs(Q_0+L_d-L_u-H_0-E_0)

            # return the residuals
            return res

        # optimise temperature in each point in the x direction

        for x in range(len(self.surf_T_now)):
            T_0 = 283. # temperature which to optimise from
            surf_args = (T_a[x],         self.surf_f[x], self.albedo[x],
                         self.surf_G[self.iter_id], self.surf_p[x], self.surf_rho[x],
                         self.surf_U[x], self.Cs_t[x],   self.Cs_q[x],
                        )
                
            res = minimize(optim_T0,x0=T_0,args=surf_args,bounds=((None,400),), \
                         method='L-BFGS-B',options={'eps':1e-8})
            
            T_0 = res.x[0]

            self.surf_T_now[x] = res.x[0]
            
        return self.surf_T_now

    def boundary_layer(self, T):
        """ Simple advection-diffusion equation.
        Nz          :: Number of grid points
        dt          :: time step in seconds
        K           :: turbulent diffusivity
        u           :: Speed of fluid
        """
        # Set BC top (Neumann condition)
        # The last term accounts for the fixed gradient of 0.01
        T[-1, :] = T[-2, :] - 0.005 * self.dz_atmo

        # Set top BC for moisture
        self.q[-1, :] = self.q[-2, :] 

        # Set BC right (Dirichlet condition)
        T[:, -1] = T[:, -2]

        # Set right BC for moisture
        self.q[:, -1] = self.q[:, -2]
        
        old_t = T
        old_q = self.q

        # First update grid cells in z-direction. Here, we loop over all x grid cells and
        # use the index arrays m, mu, md to calculate the gradients for the
        # turbulent diffusion (which only depends on z)
        for x in range(1,self.Nx-1):
            # Temperature diffusion + lapse rate 
            T[self.idx_atmo['z'],x] = T[self.idx_atmo['z'],x] + \
                ((self.atmo_K*self.dt)/(self.dz_atmo**2)) * \
                (old_t[self.idx_atmo['z_u'],x]+old_t[self.idx_atmo['z_d'],x]-2*old_t[self.idx_atmo['z'],x]) + \
                self.lapse_rate_arr[self.idx_atmo['z'],x]
            # Turbulent diffusion of moisture
            self.q[self.idx_atmo['z'],x] = self.q[self.idx_atmo['z'],x] + \
                ((self.atmo_K*self.dt)/(self.dz_atmo**2)) * (old_q[self.idx_atmo['z_u'],x]+old_q[self.idx_atmo['z_d'],x]-2*old_q[self.idx_atmo['z'],x])
            # Calculate the warming rate [K/s] by covariance
            self.cov[self.idx_atmo['z'],x] = ((self.atmo_K)/(self.dz_atmo**2)) * (old_t[self.idx_atmo['z_u'],x] + old_t[self.idx_atmo['z_d'],x] - 2*old_t[self.idx_atmo['z'],x])

        # Then update grid cells in x-direction. Here, we loop over all z grid cells and
        # use the index arrays k, kl, kr to calculate the gradients for the
        # advection (which only depends on x)
        for z in range(1,self.Nz_atmo-1):
            # temperature advection
            T[z,self.idx_len['x']] = T[z,self.idx_len['x']] - \
                ((self.u*self.dt)/(self.dx)) * \
                (old_t[z,self.idx_len['x']]-old_t[z,self.idx_len['x_l']])
            # moisture advection
            self.q[z,self.idx_len['x']] = self.q[z,self.idx_len['x']] - \
                ((self.u*self.dt)/(self.dx)) * (old_q[z,self.idx_len['x']]-old_q[z,self.idx_len['x_l']])
            
            # Calculate the warming rate [K/s] by the horizontal advection 
            # Note: Here, we use a so-called upwind-scheme (backward discretization)
            #adv[z,k] = - (u/dx)*(old[z,k]-old[z,kl])

        # Calculate new saturation mixing ratio
        self.qsat = self._mixing_ratio(T, 1013, 270, self.height)

        # Then the relative humidity using qsat
        rH = np.minimum(self.q/self.qsat, 1)

        # Correct lapse rates where rH==100% (moist adiabatic lapse rate)
        self.lapse_rate_arr[rH==1] = -0.006

        return T
    
    def heat_equation(self, T):
        # Set lower BC - Neumann condition
        T[-1, :] = T[-2, :]
            
        for x in range(self.Nx):
            # Update temperature using indices arrays
            T[self.idx_soil['z'], x] = T[self.idx_soil['z'], x] + \
                ((T[self.idx_soil['z_u'], x] - 2 * T[self.idx_soil['z'], x] + \
                T[self.idx_soil['z_d'], x])/self.dz_soil**2) * self.dt * self.soil_K

        return T
        
    def step(self):

        self.T_atmo[:,:,self.iter_id+1] = self.boundary_layer(self.T_atmo[:, :, self.iter_id])
        self.T_soil[:,:,self.iter_id+1] = self.heat_equation(self.T_soil[:, :, self.iter_id])
        
        # apply surface balance equation
        self.T_atmo[0, :, self.iter_id+1] = self.surface_balance(
            self.T_atmo[0, :, self.iter_id])
        self.T_soil[-1,:, self.iter_id+1] = self.T_atmo[0, :, self.iter_id+1]

    def run(self):

        a = perf_counter()
        
        self._init_index_arrs()
        self._init_surface()
        self._init_atmosphere()
        
        for idx in range(self.n_iters-1):
            self.iter_id = idx
            self.step()

        print(f'Model run finished in {perf_counter() - a} seconds.')
    
    def show_plot(self):
        concat_arr = np.concatenate((self.T_soil, 285*np.ones((2, self.Nx, self.n_iters)),
                                     self.T_atmo), axis=0)
        
        fig = px.imshow(np.moveaxis(concat_arr[::-1,:,:], 2, 0), animation_frame=0,
                color_continuous_scale='RdBu_r', aspect='equal', zmin=260, zmax=310)
        fig.show()

    def save_object(self, outfile):
        try:
            with open(outfile, "wb") as f:
                pickle.dump(self, f, protocol=pickle.HIGHEST_PROTOCOL)
        except Exception as ex:
            print("Error during pickling object (Possibly unsupported):", ex)
        

In [ ]:
# Model parameters
Nx = 50                      # Nx             number of simulated points in the x direction
Nz_atmo = 40                 # Nz_atmo        number of simulated points in the z direction (atmosphere)
Nz_soil = 10                 # Nz_soil        number of simulated points in the z direction (soil)
x_size = 10_000              # x_size         how long of a line does the model cover
z_size_atmo = 200            # z_size_atmo    how high does the model reach
z_size_soil = 10             # z_size_soil    how deep does the model reach
z = 2.0                      # z              height of tempearature measurement above the ground
z_0 = 1e-3                   # 
time = 3600 * 24 * 7         # time           how long does the model run [s]
dt = 60                      # dt             time step length [s]

In [ ]:
model = Combined_model(Nx, Nz_atmo, Nz_soil, x_size, z_size_atmo, z_size_soil, z, z_0, time, dt)

In [ ]:
model.run()

In [ ]:
model.show_plot()

In [ ]:
model.save_object('../results/model_7day.pickle')

In [ ]:
plt.imshow(model.T_atmo[:,:,181800]-272.15)
plt.colorbar()

In [ ]:
model_np = Combined_model_numpy(Nx, Nz_atmo, Nz_soil, x_size, z_size_atmo, z_size_soil, z, z_0, time, dt)

In [ ]:
model_np.run(verbose=True)

In [ ]:
model.run(verbose=True)

In [ ]:
fig = px.imshow(np.moveaxis(model.T_atmo[::-1,:,:], 2, 0), animation_frame=0,
                color_continuous_scale='RdBu_r', aspect='equal', zmin=280, zmax=330)
fig.show()
fig = px.imshow(np.moveaxis(model.T_soil[::-1,:,:], 2, 0), animation_frame=0,
                color_continuous_scale='RdBu_r', aspect='equal', zmin=280, zmax=330)
fig.show()

In [ ]:
data = model.T_atmo[:, :, 400]
x = np.linspace(0, x_size, Nx)
z = np.linspace(0, z_size_atmo, Nz_atmo)

print(np.unique(data))

fig, ax = plt.subplots(1,1,figsize=(18,5));
cn0 = ax.contourf(x,z,data,10,origin='lower',levels=21,cmap='RdBu_r');